In [537]:
import openpyxl


In [539]:
print(nltk.__version__)

3.9.1


In [60]:
# import necessary libraries

import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import re
import string
from string import digits
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score
import pickle
import numpy as np
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    recall_score,
)
from sklearn.model_selection import RandomizedSearchCV
from lightgbm import LGBMClassifier

In [17]:
import re
import string
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import nlpaug.augmenter.word.context_word_embs as aug
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk import word_tokenize

from sklearn.utils import shuffle
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
dataset=pd.read_excel('Dataset.xlsx')
dataset

,Claim Description,Coverage Code,Accident Source
0,THE IV WAS MAKING A LEFT TURN ON A GREEN ARROW...,AN,"Struck pedestrian, bicycle"
1,CLAIMANT ALLEGES SHE SUFFERED INJURIES IN AN E...,GB,Elevator/Escalator
2,"IV PASSENGER SUSTAINED INJURIES, OV AND IV COL...",AB,Sideswipe or lane change
3,CLAIMANT ALLEGES SHE WAS BURNED UNKNOWN DEGREE...,PA,Food Product
4,THE IV WAS MERGING INTO A CONSTRUCTION ZONE WH...,AD,Struck vehicle in rear
...,...,...,...
191685,SOME WOOD FELL OVER AND HIT GUESTS LEGS,GB,Alleged lack of security
191686,FEMALE FELL DOWN ON PLATFORM. ...,AB,Not Otherwise Classified
191687,IWAS DRIVING DOWNHILL IN CUSTOMER YARD ON WET ...,AD,Sideswipe or lane change
191688,DONNA GEORGE WAS WALKING UP TO REGISTER 2 AND ...,PM,Alleged Negligent Act


In [23]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191690 entries, 0 to 191689
Data columns (total 3 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Claim Description  191463 non-null  object
 1   Coverage Code      191690 non-null  object
 2   Accident Source    191690 non-null  object
dtypes: object(3)
memory usage: 4.4+ MB


In [24]:
dataset.isnull().sum()  #null values

Claim Description    227
Coverage Code          0
Accident Source        0
dtype: int64

In [26]:
# find duplicates in feature

dataset['Claim Description'].duplicated().sum()

27699

In [29]:
# lets check data with one of the duplicate value

dataset[dataset['Claim Description']=='CLAIMANT DROVE OVER POTHOLE CAUSING VEHICLE DAMAGE.']

,Claim Description,Coverage Code,Accident Source
296,CLAIMANT DROVE OVER POTHOLE CAUSING VEHICLE DA...,GD,Pothole
319,CLAIMANT DROVE OVER POTHOLE CAUSING VEHICLE DA...,GD,Pothole
589,CLAIMANT DROVE OVER POTHOLE CAUSING VEHICLE DA...,GD,Pothole
667,CLAIMANT DROVE OVER POTHOLE CAUSING VEHICLE DA...,GD,Pothole
763,CLAIMANT DROVE OVER POTHOLE CAUSING VEHICLE DA...,GD,Pothole
...,...,...,...
191123,CLAIMANT DROVE OVER POTHOLE CAUSING VEHICLE DA...,GD,Pothole
191265,CLAIMANT DROVE OVER POTHOLE CAUSING VEHICLE DA...,GD,Pothole
191523,CLAIMANT DROVE OVER POTHOLE CAUSING VEHICLE DA...,GD,Pothole
191550,CLAIMANT DROVE OVER POTHOLE CAUSING VEHICLE DA...,GD,Pothole


In [31]:
# drop duplicates using claim description features

dataset.drop_duplicates(subset = ['Claim Description'],inplace = True)

In [33]:
# remove null value from dataset

final_data = dataset[~dataset['Claim Description'].isna()]

final_data.shape

(163990, 3)

## Data pre-processing:

In [ ]:
nltk.download('punkt_tab')

In [40]:
def remove_stopwords(text):
    removed = []
    stop_words = list(stopwords.words("english"))
    tokens = word_tokenize(text)
    for i in range(len(tokens)):
        if tokens[i] not in stop_words:
            removed.append(tokens[i])
    return " ".join(removed)

def remove_numbers(text):
    number_pattern = r'\d+'
    without_number = re.sub(pattern=number_pattern, repl=" ", string=text)
    return without_number

def pre_processing(final_data):

    # Lowercase all characters
    final_data['Claim Description']=final_data['Claim Description'].astype(str).apply(lambda x: x.lower())
    
    final_data['Claim Description'] = final_data['Claim Description'].apply(lambda x: remove_stopwords(x))
    #final_data['Claim Description'] = final_data['Claim Description'].apply(lambda x: lemmatizing(x))
    final_data['Claim Description'] = final_data['Claim Description'].apply(lambda x: remove_numbers(x))

    final_data['Claim Description'] = final_data['Claim Description'].apply(lambda x: re.sub(r"won\'t", "will not", x))
    final_data['Claim Description'] = final_data['Claim Description'].apply(lambda x: re.sub(r"can\'t", "can not", x))

    # general
    final_data['Claim Description'] = final_data['Claim Description'].apply(lambda x: re.sub(r"n\'t", " not", x))
    final_data['Claim Description'] = final_data['Claim Description'].apply(lambda x: re.sub(r"\'re", " are", x))
    final_data['Claim Description'] = final_data['Claim Description'].apply(lambda x: re.sub(r"\'s", " is", x))
    final_data['Claim Description'] = final_data['Claim Description'].apply(lambda x: re.sub(r"\'d", " would", x))
    final_data['Claim Description'] = final_data['Claim Description'].apply(lambda x: re.sub(r"\'ll", " will", x))
    final_data['Claim Description'] = final_data['Claim Description'].apply(lambda x: re.sub(r"\'t", " not", x))
    final_data['Claim Description'] = final_data['Claim Description'].apply(lambda x: re.sub(r"\'ve", " have", x))
    final_data['Claim Description'] = final_data['Claim Description'].apply(lambda x: re.sub(r"\'m", " am", x))

    # Remove quotes
    final_data['Claim Description']=final_data['Claim Description'].apply(lambda x: re.sub("'", '', x))



    exclude = set(string.punctuation) # Set of all special characters
    # Remove all the special characters
    final_data['Claim Description']=final_data['Claim Description'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

    # remove extra
    final_data['Claim Description']=final_data['Claim Description'].apply(lambda x: re.sub('[-_.:;\[\]\|,]', '', x))


    # Remove extra spaces
    final_data['Claim Description']=final_data['Claim Description'].apply(lambda x: x.strip())

    final_data['Claim Description']=final_data['Claim Description'].apply(lambda x: re.sub(" +", " ", x))
    
    return final_data

In [41]:
final_data = pre_processing(final_data)

## class_label 1 : Coverage Code

In [49]:
# find unique classes

print('Number of classes in Coverage code :',dataset['Coverage Code'].nunique())

Number of classes in Coverage code : 43


In [50]:
final_data['Coverage Code'].value_counts()

Coverage Code
AD    43070
GB    32193
GD    22399
AP    18894
AL     9983
AB     9397
PA     8030
PB     5089
RB     4553
NS     2297
PM     1359
AU     1273
EL      855
GK      632
AN      558
RC      514
PI      384
GO      337
PL      317
DC      261
GL      237
AM      224
EP      184
IK      183
OM      138
EO      137
PP      112
TE       85
BM       66
LL       57
IM       40
BL       36
CM       30
FB       18
OI       14
RF       12
BR        6
EB        4
RQ        3
FF        3
LS        3
EI        2
PC        1
Name: count, dtype: int64

## Observation :

- There are 43 classes in Coverage code.
- There is class imabalnce observed.
- While few classes having very few data points.

### Approach 1 :

- Check classes with very few datapoints and create dictionary for features. Since 1-2 data cant be predicted
- Check classes with medium range of datapoints and convert them into one class.
- Develop model on new classes.
- For medium range classes create one more model.
- Use second model for prediction if class is in medium class.
- Finally override decision if the feature is in dictionary of very few classes.

### Approach 2 :

- Do small Data Augmentation. Since 1-2 data cant be predicted and splitted for training and testing 
- Check classes with small range of datapoints, augment them with bert to add a few rows to split it.
- Do split and Augment the train data heavily to balance the classes 
- Develop model.

### Can combine both approaches for better results, but moving forward with 2nd saving time

In [43]:
import nlpaug.augmenter.word.context_word_embs as aug

In [44]:
augmenter = aug.ContextualWordEmbsAug(model_path='bert-base-uncased', action="insert")

In [46]:
sample_text = final_data['Claim Description'].iloc[100]
sample_text 

'toilet flooded unit causing damage unit parking garage'

In [47]:
augmented_sample_text = augmenter.augment(sample_text)
augmented_sample_text

['toilet flooded bus unit causing considerable damage unit parking facility garage']

In [51]:
# check count of classes

coverage_code_classes = pd.DataFrame(final_data['Coverage Code'].value_counts()) #created dataset of value count
coverage_code_classes

,count
Coverage Code,
AD,43070
GB,32193
GD,22399
AP,18894
AL,9983
AB,9397
PA,8030
PB,5089
RB,4553


In [86]:
coverage_code_classes[coverage_code_classes['count'] < 50].index.to_list()

['IM', 'BL', 'CM', 'FB', 'OI', 'RF', 'BR', 'EB', 'RQ', 'FF', 'LS', 'EI', 'PC']

In [88]:
print(len(coverage_code_classes[coverage_code_classes['count'] < 2000].index.to_list()))
coverage_code_classes[coverage_code_classes['count'] < 2000].index.to_list()

33


['PM',
 'AU',
 'EL',
 'GK',
 'AN',
 'RC',
 'PI',
 'GO',
 'PL',
 'DC',
 'GL',
 'AM',
 'EP',
 'IK',
 'OM',
 'EO',
 'PP',
 'TE',
 'BM',
 'LL',
 'IM',
 'BL',
 'CM',
 'FB',
 'OI',
 'RF',
 'BR',
 'EB',
 'RQ',
 'FF',
 'LS',
 'EI',
 'PC']

In [103]:
very_few_classes = coverage_code_classes[coverage_code_classes['count'] < 50].index.to_list()
medium_classes = coverage_code_classes[coverage_code_classes['count'] < 2000].index.to_list()

In [66]:
# check the data with very few classes

fewer_data = final_data[final_data['Coverage Code'].isin(very_few_classes)]
print(fewer_data.shape)
fewer_data.head(2)

(172, 3)


,Claim Description,Coverage Code,Accident Source
803,property loss,IM,Not Otherwise Classified
832,wind caused tractor trailer roll potential inj...,BL,Windstorm


In [58]:
# for fewer classes either we can directrly create dictionary and use classes


fewer_class_dictionary = dict()
for idx,row in fewer_data.iterrows():
    
    fewer_class_dictionary[row[0]] = [row[1],row[2]]

print(fewer_class_dictionary[fewer_data['Claim Description'].iloc[0]])
fewer_class_dictionary

['BR', 'Burglary, Robbery, Theft - Property']


{'caller states builder called stated items property used build home stolen police report created': ['BR',
  'Burglary, Robbery, Theft - Property'],
 'oil discovered monitoring tanks': ['EI', 'Fuel'],
 'eeoc charge discrimination filed current st john university employee charles fortmann': ['EB',
  'Alleged Discrimination'],
 'allegations violation wage hour violations limited assignment payment legal fees': ['EB',
  'Alleged Negligent Act'],
 'graffiti painted wall': ['RQ', 'Other - comprehensive'],
 'sewage release': ['EI', 'Backup Sewer, Seepage'],
 'cyber breach': ['FF', 'Not Otherwise Classified'],
 'claimant suffered defamation loss business': ['LS',
  'Loss of Use, Business Interruption'],
 'property theft damage': ['BR', 'Burglary, Robbery, Theft - Property'],
 'tools stolen job site francis howell north': ['BR',
  'Burglary, Robbery, Theft - Property'],
 'earthquake': ['RQ', 'Rain'],
 'claimant suffered abuse attack': ['LS', 'Assault and/or abuse'],
 'claimant alleging theft a

In [61]:
# save dictionary
path = "results/fewer_class_dictionary.pickle"
pickle.dump(fewer_class_dictionary, open(path, "wb"))
print("Model saved to", path)

Model saved to results/fewer_class_dictionary.pickle


## Augment medium range data

In [63]:
final_data['Coverage Code'].value_counts()

Coverage Code
AD    43070
GB    32193
GD    22399
AP    18894
AL     9983
AB     9397
PA     8030
PB     5089
RB     4553
NS     2297
PM     1359
AU     1273
EL      855
GK      632
AN      558
RC      514
PI      384
GO      337
PL      317
DC      261
GL      237
AM      224
EP      184
IK      183
OM      138
EO      137
PP      112
TE       85
BM       66
LL       57
IM       40
BL       36
CM       30
FB       18
OI       14
RF       12
BR        6
EB        4
RQ        3
FF        3
LS        3
EI        2
PC        1
Name: count, dtype: int64

In [ ]:
def augmentMyData(df, augmenter, repetitions=1, samples=200):
    augmented_texts = []
    # select only the minority class samples
    spam_df = df[df['label'] == 1].reset_index(drop=True) # removes unecessary index column
    for i in tqdm(np.random.randint(0, len(spam_df), samples)):
        # generating 'n_samples' augmented texts
        for _ in range(repetitions):
            augmented_text = augmenter.augment(spam_df['text'].iloc[i])
            augmented_texts.append(augmented_text)
    
    data = {
        'label': 1,
        'text': augmented_texts
    }
    aug_df = pd.DataFrame(data)
    df = shuffle(df.append(aug_df).reset_index(drop=True))
    return df

In [77]:
for i in medium_classes:
    print(i)

NS
PM
AU
EL
GK
AN
RC
PI
GO
PL
DC
GL
AM
EP
IK
OM
EO
PP
TE
BM
LL
IM
BL
CM
FB
OI
RF
BR
EB
RQ
FF
LS
EI
PC


In [82]:
final_data.shape[0]

163990

In [79]:
final_data.head(2)

,Claim Description,Coverage Code,Accident Source
0,iv making left turn green arrow pedestrian ran...,AN,"Struck pedestrian, bicycle"
1,claimant alleges suffered injuries elevator,GB,Elevator/Escalator


In [95]:
print(len(medium_classes))
medium_classes

34


['NS',
 'PM',
 'AU',
 'EL',
 'GK',
 'AN',
 'RC',
 'PI',
 'GO',
 'PL',
 'DC',
 'GL',
 'AM',
 'EP',
 'IK',
 'OM',
 'EO',
 'PP',
 'TE',
 'BM',
 'LL',
 'IM',
 'BL',
 'CM',
 'FB',
 'OI',
 'RF',
 'BR',
 'EB',
 'RQ',
 'FF',
 'LS',
 'EI',
 'PC']

In [104]:
very_few_classes

['IM', 'BL', 'CM', 'FB', 'OI', 'RF', 'BR', 'EB', 'RQ', 'FF', 'LS', 'EI', 'PC']

In [106]:
medium_class_without_few_class= []
for class_label in medium_classes:
    if class_label in very_few_classes:
        continue
        
    else:
        medium_class_without_few_class.append(class_label)
        
medium_class_without_few_class

['PM',
 'AU',
 'EL',
 'GK',
 'AN',
 'RC',
 'PI',
 'GO',
 'PL',
 'DC',
 'GL',
 'AM',
 'EP',
 'IK',
 'OM',
 'EO',
 'PP',
 'TE',
 'BM',
 'LL']

In [321]:
final_data = pd.read_excel('augmented_data.xlsx') # I got this from Oversampling you will understand it below

In [322]:
final_data['Coverage Code'].value_counts()

Coverage Code
AD    43070
GB    32193
GD    22399
AP    18894
AL     9983
AB     9397
PA     8030
PB     5089
RB     4553
NS     2297
PM     1859
AU     1773
EL     1355
GK     1132
AN     1058
RC     1014
PI      884
GO      837
PL      817
DC      761
GL      737
AM      724
EP      684
IK      683
OM      638
EO      637
PP      612
TE      585
BM      566
LL      557
IM       40
BL       36
CM       30
FB       18
OI       14
RF       12
BR        6
EB        4
LS        3
FF        3
RQ        3
EI        2
PC        1
Name: count, dtype: int64

In [323]:
final_data

,Claim Description,Coverage Code
0,iv traveling wbnd desert inn rd east pawnee dr...,AB
1,claimant slipped stairs causing bodily injury,GB
2,customer driving parking lot cart came cart co...,GD
3,loading guest forklift pallet made contact rea...,GD
4,injuries mirror strike sideswiped another vehicle,AP
...,...,...
173985,reginal manager following another employee hom...,AP
173986,iv backed parked,AD
173987,iv hit parked unknown party,AP
173988,stray bullet fired unknown persons struck hvac...,RB


In [324]:
final_data.drop(labels='Accident Source',axis=1)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\Admin\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py", line 3550, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_10532\981238603.py", line 1, in <module>
    final_data.drop(labels='Accident Source',axis=1)
  File "C:\Users\Admin\anaconda3\lib\site-packages\pandas\core\frame.py", line 5344, in drop
    axes_single_arg=_shared_doc_kwargs["axes_single_arg"],
  File "C:\Users\Admin\anaconda3\lib\site-packages\pandas\core\generic.py", line 4711, in drop
    )
  File "C:\Users\Admin\anaconda3\lib\site-packages\pandas\core\generic.py", line 4753, in _drop_axis
    errors: IgnoreRaise = ...,
  File "C:\Users\Admin\anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 7000, in drop
    # dt64/td64 to integer, also about treating tuples as sequences
KeyError: "['Accident Source'] not found in axis"

During handling of the above excep

In [325]:
# Initialize the augmenter
#augmenter = naw.ContextualWordEmbsAug(model_path='bert-base-uncased', action="insert")

def augment_data_for_selected_classes(df, selected_classes, samples=200, repetitions=1):
    """
    Augments data for selected classes in a DataFrame.
    
    Parameters:
        df (pd.DataFrame): Input DataFrame with columns 'label' and 'text'.
        selected_classes (list): List of classes to augment.
        samples (int): Number of samples to select for each class.
        repetitions (int): Number of augmented examples to generate per sample.
        
    Returns:
        pd.DataFrame: DataFrame with augmented data added.
    """
    augmented_texts = []
    augmented_labels = []
    
    # Loop through each selected class
    for class_label in selected_classes:
        print(f"Augmenting class {class_label}...")
        
        # Filter for the current class
        class_df = df[df['Coverage Code'] == class_label].reset_index(drop=True)
        
        # Randomly sample from the class
        indices = np.random.choice(len(class_df), samples, replace=True)
        
        # Augment selected samples
        for idx in tqdm(indices, desc=f"Class {class_label}"):
            original_text = class_df['Claim Description'].iloc[idx]
            for _ in range(repetitions):
                augmented_text = augmenter.augment(original_text)
                augmented_texts.append(augmented_text)
                augmented_labels.append(class_label)
    
    # Create a DataFrame for augmented data
    augmented_df = pd.DataFrame({'Coverage Code': augmented_labels, 'Claim Description': augmented_texts})
    
    # Combine original and augmented data, shuffle, and reset index
    df = shuffle(pd.concat([df, augmented_df], ignore_index=True))
    return df
'''
# Example usage
if __name__ == "__main__":
    # Example dataset
    data = {
        'label': np.random.randint(0, 40, 1000),  # 40 classes
        'text': ["Example text data"] * 1000      # Dummy text
    }
    df = pd.DataFrame(data)
    
    # Select 10 classes for augmentation
    selected_classes = np.random.choice(range(40), 10, replace=False)  # Randomly pick 10 classes
    
    # Augment the data
    augmented_df = augment_data_for_selected_classes(df, selected_classes, samples=200, repetitions=1)
    
    augmented_df = augment_data_for_selected_classes(final_data.drop(labels='Accident Source',axis=1), medium_class_without_few_class, samples=20, repetitions=1)
    # Show augmented dataset
    print(augmented_df.head())
'''

'\n# Example usage\nif __name__ == "__main__":\n    # Example dataset\n    data = {\n        \'label\': np.random.randint(0, 40, 1000),  # 40 classes\n        \'text\': ["Example text data"] * 1000      # Dummy text\n    }\n    df = pd.DataFrame(data)\n    \n    # Select 10 classes for augmentation\n    selected_classes = np.random.choice(range(40), 10, replace=False)  # Randomly pick 10 classes\n    \n    # Augment the data\n    augmented_df = augment_data_for_selected_classes(df, selected_classes, samples=200, repetitions=1)\n    \n    augmented_df = augment_data_for_selected_classes(final_data.drop(labels=\'Accident Source\',axis=1), medium_class_without_few_class, samples=20, repetitions=1)\n    # Show augmented dataset\n    print(augmented_df.head())\n'

In [327]:
augmented_df = augment_data_for_selected_classes(final_data, medium_classes, samples=5, repetitions=1)
# Show augmented dataset
print(augmented_df.head())

Augmenting class PM...


Class PM:   0%|          | 0/5 [00:00<?, ?it/s]

Augmenting class AU...


Class AU:   0%|          | 0/5 [00:00<?, ?it/s]

Augmenting class EL...


...

Augmenting class EI...


Class EI:   0%|          | 0/5 [00:00<?, ?it/s]

Augmenting class PC...


Class PC:   0%|          | 0/5 [00:00<?, ?it/s]

                                        Claim Description Coverage Code
84769                     client driver reversed claimant            AD
146749  clmt alleges receiving coffee dt lid coffee po...            PA
128996  jackie claims pushing rack caught right ring f...            EL
72183   guest alledges pump failed stop flowing fuel c...            GD
120718  cart hit guests car due wind guests vehicle de...            GD


In [328]:
augmented_df['Coverage Code'].value_counts() # new data i imported have 500 augmented row in medium class except fewer class

Coverage Code
AD    43070
GB    32193
GD    22399
AP    18894
AL     9983
AB     9397
PA     8030
PB     5089
RB     4553
NS     2297
PM     1864
AU     1778
EL     1360
GK     1137
AN     1063
RC     1019
PI      889
GO      842
PL      822
DC      766
GL      742
AM      729
EP      689
IK      688
OM      643
EO      642
PP      617
TE      590
BM      571
LL      562
IM       45
BL       41
CM       35
FB       23
OI       19
RF       17
BR       11
EB        9
RQ        8
FF        8
LS        8
EI        7
PC        6
Name: count, dtype: int64

In [128]:
print("Original: ", final_data.shape)
print("Augmented: ", augmented_df.shape)  #there are 33 coverage code in medium class and we augmented 5 row in each

Original:  (163990, 3)
Augmented:  (164010, 2)


### Notice the value count of medium classes has increased 
#### I ran the same code for 500 samples in another collab while moving forward in this notebook
- Will merge later

##### PS : I have merged it above

In [139]:
#augmented_df.to_excel('final1_data.xlsx', index=False, engine="xlsxwriter")

TypeError: write() got an unexpected keyword argument 'engine_kwargs'

In [329]:
augmented_df.head(1)

,Claim Description,Coverage Code
84769,client driver reversed claimant,AD


### Under Sampling

In [359]:
from imblearn.under_sampling import RandomUnderSampler

# Randomly under sample the majority class
rus = RandomUnderSampler(random_state=42, sampling_strategy={'AD':10000,'GB':10000,'GD':10000,'AP':10000})
df_train_rus, _ = rus.fit_resample(augmented_df, augmented_df['Coverage Code'])

In [360]:
df_train_rus['Coverage Code'].value_counts()

Coverage Code
AP    10000
AD    10000
GD    10000
GB    10000
AL     9983
AB     9397
PA     8030
PB     5089
RB     4553
NS     2297
PM     1864
AU     1778
EL     1360
GK     1137
AN     1063
RC     1019
PI      889
GO      842
PL      822
DC      766
GL      742
AM      729
EP      689
IK      688
OM      643
EO      642
PP      617
TE      590
BM      571
LL      562
IM       45
BL       41
CM       35
FB       23
OI       19
RF       17
BR       11
EB        9
LS        8
FF        8
RQ        8
EI        7
PC        6
Name: count, dtype: int64

# Cant do train test split with one value count of a class we need atleast 2
## Thats why did a small oversampling now performing oversampling on large scale

In [424]:
# split the dataset before augmenting to avoid augmented data in valid set
X= df_train_rus['Claim Description']
y=  df_train_rus['Coverage Code']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.1,stratify=y,random_state=33)

In [425]:
X_test.shape

(9760,)

#### RandomOverSampler

In [426]:
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

In [427]:
Counter(augmented_df['Coverage Code'])

Counter({'AD': 43070,
         'GB': 32193,
         'GD': 22399,
         'AP': 18894,
         'AL': 9983,
         'AB': 9397,
         'PA': 8030,
         'PB': 5089,
         'RB': 4553,
         'NS': 2297,
         'PM': 1864,
         'AU': 1778,
         'EL': 1360,
         'GK': 1137,
         'AN': 1063,
         'RC': 1019,
         'PI': 889,
         'GO': 842,
         'PL': 822,
         'DC': 766,
         'GL': 742,
         'AM': 729,
         'EP': 689,
         'IK': 688,
         'OM': 643,
         'EO': 642,
         'PP': 617,
         'TE': 590,
         'BM': 571,
         'LL': 562,
         'IM': 45,
         'BL': 41,
         'CM': 35,
         'FB': 23,
         'OI': 19,
         'RF': 17,
         'BR': 11,
         'EB': 9,
         'RQ': 8,
         'FF': 8,
         'LS': 8,
         'EI': 7,
         'PC': 6})

In [428]:
pd.DataFrame(X_train).astype(str)

,Claim Description
63085,customer reported marvin truck hit card reader...
144128,calcium leaks damaged clmnt is vehicle
77431,['subject matter elizabeth kolar security brea...
6748,wife opened caesar salad ate lunch rd th march...
95620,customer entering store door caught back flip ...
...,...
145169,tree fell across one apartments causing roof d...
145626,guest fell shower injured
17103,gfs backing hit corner building trailer bricks...
75866,iv leaving parking lot commercial account pick...


### Vectorization

In [429]:
type(X_train)

pandas.core.series.Series

In [430]:
print(type(X_train[0]))

if type(X_train[3]) != str:
    print(1)
else: print(0)

<class 'str'>
0


In [431]:
X_train

63085     customer reported marvin truck hit card reader...
144128               calcium leaks damaged clmnt is vehicle
77431     ['subject matter elizabeth kolar security brea...
6748      wife opened caesar salad ate lunch rd th march...
95620     customer entering store door caught back flip ...
                                ...                        
145169    tree fell across one apartments causing roof d...
145626                            guest fell shower injured
17103     gfs backing hit corner building trailer bricks...
75866     iv leaving parking lot commercial account pick...
38687     iv pulling forward parking spot cut tight stri...
Name: Claim Description, Length: 87838, dtype: object

In [432]:
X_train[0]

'iv traveling wbnd desert inn rd east pawnee dr vehicle directly front toyota camry ingrid ibarra hit brakes forcefully coming almost complete stop effort avoid collision iv swerved left unoccupied centeremergency lane unsuccessful struck cv directly front presumably caused thecv toyota enter lane causing chain reaction resulted ford focus'

In [446]:
df_train_rus[df_train_rus.index==21026]

,Claim Description,Coverage Code
21026,small pd iv backing onto road iv bed flipped i...,AL


In [442]:
count = 0
l = []
for i in range(0,87838):
    if type(X_train.iloc[i]) != str:
        print(count,'-->',type(X_train.iloc[i]))
        l.append(count)
    count = count+1

71 --> <class 'list'>
2404 --> <class 'list'>
2658 --> <class 'list'>
3089 --> <class 'list'>
4313 --> <class 'list'>
4643 --> <class 'list'>
4950 --> <class 'list'>
4962 --> <class 'list'>
5835 --> <class 'list'>
5972 --> <class 'list'>
6117 --> <class 'list'>
6688 --> <class 'list'>
6966 --> <class 'list'>
7220 --> <class 'list'>
7249 --> <class 'list'>
7321 --> <class 'list'>
8048 --> <class 'list'>
8336 --> <class 'float'>
8351 --> <class 'list'>
9581 --> <class 'list'>
9861 --> <class 'list'>
10011 --> <class 'list'>
10327 --> <class 'list'>
10522 --> <class 'list'>
11925 --> <class 'list'>
12201 --> <class 'list'>
12593 --> <class 'list'>
13235 --> <class 'list'>
14839 --> <class 'list'>
15264 --> <class 'list'>
15432 --> <class 'list'>
16614 --> <class 'list'>
17033 --> <class 'list'>
18221 --> <class 'list'>
18693 --> <class 'list'>
19323 --> <class 'list'>
19481 --> <class 'list'>
19896 --> <class 'list'>
20413 --> <class 'list'>
20793 --> <class 'list'>
20834 --> <class 'list

In [444]:
print(len(l))
l

149


[71,
 2404,
 2658,
 3089,
 4313,
 4643,
 4950,
 4962,
 5835,
 5972,
 6117,
 6688,
 6966,
 7220,
 7249,
 7321,
 8048,
 8336,
 8351,
 9581,
 9861,
 10011,
 10327,
 10522,
 11925,
 12201,
 12593,
 13235,
 14839,
 15264,
 15432,
 16614,
 17033,
 18221,
 18693,
 19323,
 19481,
 19896,
 20413,
 20793,
 20834,
 21026,
 21441,
 23414,
 24359,
 24839,
 25461,
 25701,
 26372,
 26403,
 28931,
 28981,
 29508,
 29719,
 29817,
 30317,
 30839,
 31235,
 31589,
 33352,
 33525,
 34499,
 35816,
 36354,
 36764,
 37330,
 37595,
 37785,
 38895,
 39505,
 40353,
 40385,
 42032,
 42532,
 43425,
 43734,
 45229,
 45258,
 45370,
 45457,
 45646,
 45809,
 46465,
 46557,
 47574,
 48518,
 48772,
 49220,
 49672,
 50358,
 52155,
 52663,
 54145,
 54246,
 54494,
 54941,
 55842,
 56415,
 56471,
 56674,
 57068,
 59406,
 60020,
 60172,
 60461,
 60563,
 60892,
 61195,
 61606,
 62145,
 62167,
 62723,
 63184,
 64245,
 64568,
 65507,
 65646,
 66056,
 67896,
 68258,
 68276,
 68279,
 68991,
 69330,
 72848,
 73081,
 74248,
 74433,

In [439]:
X_train

63085     customer reported marvin truck hit card reader...
144128               calcium leaks damaged clmnt is vehicle
77431     ['subject matter elizabeth kolar security brea...
6748      wife opened caesar salad ate lunch rd th march...
95620     customer entering store door caught back flip ...
                                ...                        
145169    tree fell across one apartments causing roof d...
145626                            guest fell shower injured
17103     gfs backing hit corner building trailer bricks...
75866     iv leaving parking lot commercial account pick...
38687     iv pulling forward parking spot cut tight stri...
Name: Claim Description, Length: 87838, dtype: object

In [388]:
# convert data into vector form

'''
count_vector_step_1 = CountVectorizer()

X_traintf = count_vector_step_1.fit_transform(X_train)
X_testtf = count_vector_step_1.transform(X_test)

'''

AttributeError: 'list' object has no attribute 'lower'

In [447]:
# convert data into vector form

count_vector_step_1 = CountVectorizer()

X_train = count_vector_step_1.fit_transform(X_train)
X_test = count_vector_step_1.transform(X_test)

# Initialize CountVectorizer
count_vector_step_1 = CountVectorizer()

# Fit and transform the training data
X_traintf = count_vector_step_1.fit_transform(X_train)

# Transform the test data
X_testtf = count_vector_step_1.transform(X_test)

In [448]:
X_train.shape

(87838,)

In [449]:
X_traintf.shape

(87838, 55074)

In [450]:
y_train

63085     AD
144128    GD
77431     GO
6748      PA
95620     GB
          ..
145169    RB
145626    PM
17103     AD
75866     AB
38687     AP
Name: Coverage Code, Length: 87838, dtype: object

In [451]:
augmented_df

,Claim Description,Coverage Code
84769,client driver reversed claimant,AD
146749,clmt alleges receiving coffee dt lid coffee po...,PA
128996,jackie claims pushing rack caught right ring f...,EL
72183,guest alledges pump failed stop flowing fuel c...,GD
120718,cart hit guests car due wind guests vehicle de...,GD
...,...,...
147210,driver parked exiting vehicle saw utility vehi...,AD
61289,claimants town thanksgiving holiday received n...,PB
161522,small pd driver picking yard roll slid close b...,AL
28482,v door properly latched drove passed v road do...,AD


In [452]:
df_train_rus['Coverage Code'].value_counts()

Coverage Code
AP    10000
AD    10000
GD    10000
GB     9999
AL     9983
AB     9397
PA     8030
PB     5089
RB     4553
NS     2297
PM     1864
AU     1778
EL     1360
GK     1137
AN     1063
RC     1019
PI      889
GO      842
PL      822
DC      766
GL      742
AM      729
EP      689
IK      688
OM      643
EO      642
PP      617
TE      590
BM      571
LL      562
IM       45
BL       41
CM       35
FB       23
OI       19
RF       17
BR       11
EB        9
LS        8
FF        8
RQ        8
EI        7
PC        6
Name: count, dtype: int64

In [469]:
df_train_rus['Coverage Code'].value_counts()[9:-13].index

Index(['NS', 'PM', 'AU', 'EL', 'GK', 'AN', 'RC', 'PI', 'GO', 'PL', 'DC', 'GL',
       'AM', 'EP', 'IK', 'OM', 'EO', 'PP', 'TE', 'BM', 'LL'],
      dtype='object', name='Coverage Code')

### Oversampling for balancing

In [454]:
pd.DataFrame(X_testtf)

,0
0,"(0, 10682)\t1\n (0, 12664)\t1\n (0, 24209)..."
1,"(0, 362)\t1\n (0, 8155)\t1\n (0, 8213)\t1\..."
2,"(0, 5074)\t1\n (0, 9736)\t1\n (0, 9819)\t1..."
3,"(0, 1394)\t1\n (0, 4048)\t1\n (0, 4361)\t1..."
4,"(0, 10434)\t1\n (0, 14449)\t1\n (0, 22582)..."
...,...
9755,"(0, 6776)\t1\n (0, 10232)\t1\n (0, 37674)\..."
9756,"(0, 2772)\t1\n (0, 2898)\t1\n (0, 4873)\t1..."
9757,"(0, 2256)\t1\n (0, 2439)\t1\n (0, 3889)\t1..."
9758,"(0, 17357)\t1\n (0, 18065)\t1\n (0, 18326)..."


In [455]:
y_train

63085     AD
144128    GD
77431     GO
6748      PA
95620     GB
          ..
145169    RB
145626    PM
17103     AD
75866     AB
38687     AP
Name: Coverage Code, Length: 87838, dtype: object

In [ ]:
#df_train_rus['Coverage Code']  = y_train

In [470]:
# Check the class distribution before oversampling
print("Before SMOTE:")
print(pd.Series(y_train).value_counts())
dic = {'NS':3000, 'PM':3000, 'AU':3000, 'EL':3000, 'GK':3000, 'AN':3000, 'RC':3000,
       'PI':3000, 'GO':3000, 'PL':3000, 'DC':3000, 'GL':3000,
       'AM':3000, 'EP':3000, 'IK':3000, 'OM':3000, 'EO':3000, 'PP':3000, 'TE':3000, 'BM':3000, 'LL':3000,
       'IM':500, 'BL':500, 'CM':500, 'FB':500, 'OI':500, 'RF':500, 
       'BR':500, 'EB':500, 'LS':500, 'FF':500, 'RQ':500, 'EI':500,'PC':500}
# Perform SMOTE oversampling on the training set
smote = SMOTE(sampling_strategy= dic, random_state=42,k_neighbors=4)

X_train_resampled, y_train_resampled = smote.fit_resample(X_traintf,y_train)

# Check the class distribution after oversampling
print("\nAfter SMOTE:")
print(pd.Series(y_train_resampled).value_counts())

Before SMOTE:
Coverage Code
AD    9000
AP    9000
GD    9000
GB    8999
AL    8985
AB    8457
PA    7227
PB    4580
RB    4098
NS    2067
PM    1678
AU    1600
EL    1224
GK    1023
AN     957
RC     917
PI     800
GO     758
PL     740
DC     689
GL     668
AM     656
EP     620
IK     619
OM     579
EO     578
PP     555
TE     531
BM     514
LL     506
IM      41
BL      37
CM      32
FB      21
OI      17
RF      15
BR      10
EB       8
LS       7
RQ       7
FF       7
EI       6
PC       5
Name: count, dtype: int64

After SMOTE:
Coverage Code
AD    9000
AP    9000
GD    9000
GB    8999
AL    8985
AB    8457
PA    7227
PB    4580
RB    4098
PL    3000
BM    3000
NS    3000
PP    3000
RC    3000
AN    3000
PI    3000
AM    3000
EO    3000
DC    3000
GL    3000
TE    3000
EL    3000
IK    3000
PM    3000
AU    3000
LL    3000
EP    3000
GK    3000
GO    3000
OM    3000
RQ     500
PC     500
BR     500
OI     500
EB     500
FF     500
CM     500
RF     500
FB     500
LS     500
IM   

In [480]:
type(X_train_resampled)

scipy.sparse._csr.csr_matrix

In [472]:
X_train_resampled.shape

(138846, 55074)

In [473]:
y_train_resampled.shape

(138846,)

In [482]:


# train
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train_resampled, y_train_resampled)
print("Model trained")

# save model
path = "results/nb.pickle"
pickle.dump(naive_bayes, open(path, "wb"))
print("Model saved to", path)

# predict
y_pred = naive_bayes.predict(X_testtf)

accuracy = accuracy_score(y_test, y_pred)
print('Accuracy for naive bayes :',np.round(accuracy,2))

print("*"*100)

target_names = naive_bayes.classes_
print(classification_report(y_test, y_pred, target_names=target_names))
print('*'*100)

Model trained
Model saved to results/nb.pickle
Accuracy for naive bayes : 0.68
****************************************************************************************************
              precision    recall  f1-score   support

          AB       0.54      0.58      0.56       940
          AD       0.49      0.43      0.46      1000
          AL       0.80      0.96      0.87       998
          AM       0.50      0.38      0.43        73
          AN       0.81      0.21      0.33       106
          AP       0.50      0.57      0.53      1000
          AU       0.63      0.58      0.60       178
          BL       0.00      0.00      0.00         4
          BM       0.72      0.86      0.78        57
          BR       0.00      0.00      0.00         1
          CM       0.00      0.00      0.00         3
          DC       0.86      0.66      0.75        77
          EB       0.00      0.00      0.00         1
          EI       1.00      1.00      1.00         1
         

In [483]:
from sklearn.linear_model import LogisticRegression

lr_basemodel =LogisticRegression()

lr_basemodel.fit(X_train_resampled, y_train_resampled)
print("Model trained")

# save model
path = "results/logreg.pickle"
pickle.dump(lr_basemodel, open(path, "wb"))
print("Model saved to", path)

# predict
y_pred = lr_basemodel.predict(X_testtf)

accuracy = accuracy_score(y_test, y_pred)
print('Accuracy for logistic regression :',np.round(accuracy,2))

print("*"*100)

target_names = lr_basemodel.classes_
print(classification_report(y_test, y_pred, target_names=target_names))
print('*'*100)

Model trained
Model saved to results/logreg.pickle
Accuracy for logistic regression : 0.72
****************************************************************************************************
              precision    recall  f1-score   support

          AB       0.59      0.60      0.59       940
          AD       0.57      0.40      0.47      1000
          AL       0.95      0.95      0.95       998
          AM       0.42      0.62      0.50        73
          AN       0.37      0.50      0.43       106
          AP       0.57      0.62      0.59      1000
          AU       0.65      0.67      0.66       178
          BL       0.14      0.25      0.18         4
          BM       0.77      0.98      0.86        57
          BR       0.17      1.00      0.29         1
          CM       0.00      0.00      0.00         3
          DC       0.74      0.87      0.80        77
          EB       0.00      0.00      0.00         1
          EI       1.00      1.00      1.00        

In [485]:
## Random forest

from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier(max_depth=110).fit(X_train_resampled,y_train_resampled)
y_pred=classifier.predict(X_testtf)
print(classification_report(y_test,y_pred))

print("*"*100)
# save model
path = "results/randfrst.pickle"
pickle.dump(classifier, open(path, "wb"))
print("Model saved to", path)

              precision    recall  f1-score   support

          AB       0.60      0.58      0.59       940
          AD       0.52      0.47      0.50      1000
          AL       0.89      0.97      0.93       998
          AM       0.88      0.81      0.84        73
          AN       0.84      0.63      0.72       106
          AP       0.53      0.58      0.55      1000
          AU       0.78      0.70      0.74       178
          BL       0.67      0.50      0.57         4
          BM       0.88      1.00      0.93        57
          BR       0.50      1.00      0.67         1
          CM       0.00      0.00      0.00         3
          DC       0.83      0.95      0.88        77
          EB       0.00      0.00      0.00         1
          EI       1.00      1.00      1.00         1
          EL       0.84      0.62      0.72       136
          EO       0.89      0.98      0.93        64
          EP       0.87      0.99      0.93        69
          FB       0.07    

## class_label 2 : Accident Source

In [487]:
# find unique classes

print('Number of classes in Coverage code :',dataset['Accident Source'].nunique())

Number of classes in Coverage code : 312


In [488]:
dataset['Accident Source'].value_counts()

Accident Source
Alleged Negligent Act            20254
Sideswipe or lane change         15414
Struck animal or object           7720
Struck vehicle in rear            7401
Not Otherwise Classified          7377
                                 ...  
Multiple Vehicle Collision          14
Machine - point of operation        14
VEHICLE                             13
Defective Pipework                  11
Boiler, pressure vessel, etc.       10
Name: count, Length: 312, dtype: int64

## Observation :

- There are 312 classes in Coverage code.
- There is class imabalnce observed.
- While few classes having very few data points.

# Dont have much time left for case study and steps are similar, so directly training model for Accident Source

In [496]:
final_data2 = pre_processing(dataset)

In [497]:
final_data2

,Claim Description,Coverage Code,Accident Source
0,iv making left turn green arrow pedestrian ran...,AN,"Struck pedestrian, bicycle"
1,claimant alleges suffered injuries elevator,GB,Elevator/Escalator
2,iv passenger sustained injuries ov iv collided...,AB,Sideswipe or lane change
3,claimant alleges burned unknown degree hot tea...,PA,Food Product
4,iv merging construction zone rear ended theov ...,AD,Struck vehicle in rear
...,...,...,...
191683,package car leaked oil concrete driveway,AD,Oil or grease
191685,wood fell hit guests legs,GB,Alleged lack of security
191686,female fell platform bus reg bk ywl,AB,Not Otherwise Classified
191687,iwas driving downhill customer yard wet slippe...,AD,Sideswipe or lane change


In [498]:
# create features and class label

X = final_data2['Claim Description']

y = final_data2['Accident Source']

# split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify=y)

In [499]:
# convert data into vector form

count_vectorizer_2 = CountVectorizer()

X_train = count_vectorizer_2.fit_transform(X_train)
X_test = count_vectorizer_2.transform(X_test)

In [500]:
# train
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train, y_train)
print("Model trained")

# save model
path = "results/nb2.pickle"
pickle.dump(naive_bayes, open(path, "wb"))
print("Model saved to", path)

# predict
y_pred = naive_bayes.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print('Accuracy for naive bayes :',np.round(accuracy,2))

print("*"*100)

target_names = naive_bayes.classes_
print(classification_report(y_test, y_pred, target_names=target_names))
print('*'*100)

Model trained
Model saved to results/nb2.pickle
Accuracy for naive bayes : 0.36
****************************************************************************************************
                                            precision    recall  f1-score   support

    Accidental disposal of property/object       0.00      0.00      0.00        98
                   Administrative Decision       0.00      0.00      0.00        23
                                  Aircraft       0.00      0.00      0.00         3
                    Alleged Discrimination       0.00      0.00      0.00        20
                      Alleged False Arrest       0.00      0.00      0.00         8
                          Alleged Incident       0.00      0.00      0.00        21
      Alleged Misconduct - Personal Injury       0.00      0.00      0.00        22
                     Alleged Negligent Act       0.27      0.82      0.40      4051
         Alleged Violation of Civil Rights       0.00      0.0

#### Using bert for embedding :

In [502]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [504]:
X

array(['iv traveling wbnd desert inn rd east pawnee dr vehicle directly front toyota camry ingrid ibarra hit brakes forcefully coming almost complete stop effort avoid collision iv swerved left unoccupied centeremergency lane unsuccessful struck cv directly front presumably caused thecv toyota enter lane causing chain reaction resulted ford focus',
       'claimant slipped stairs causing bodily injury',
       'customer driving parking lot cart came cart coral struck vehicle',
       ..., 'iv hit parked unknown party',
       'stray bullet fired unknown persons struck hvac unit',
       'claimant alleges vehicle damaged pothole parking space'],
      dtype=object)

In [503]:
X = final_data['Claim Description'].values
y = final_data['Accident Source']

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\Admin\anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 3791, in get_loc
    >>> unique_index = pd.Index(list('abc'))
  File "index.pyx", line 152, in pandas._libs.index.IndexEngine.get_loc
  File "index.pyx", line 181, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 7080, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 7088, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'Accident Source'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\Admin\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py", line 3550, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_10532\2884066349.py", line 2, in <module>
    y = final_data['Accident Source'

In [505]:
X_bert_enc = model.encode(X, batch_size=500, show_progress_bar=True,)
print(X_bert_enc.shape)

Batches:   0%|          | 0/348 [00:00<?, ?it/s]

(173990, 384)


In [509]:
pickle.dump(count_vector_step_1, open("./vectorizer/count_vector_step_1.pickel","wb"))
pickle.dump(count_vectorizer_2, open("./vectorizer/count_vector_step_2.pickel","wb"))

# pickle.dump(count_vector_as_step_1, open("./vectorizer/count_vector_as_step_1.pickel","wb"))

### outputs needed 

- randfrst.pickle ---  COverage Code predictions
- nb2.pickle        ---  Accident Source prediction
- |
- |
- X= df_train_rus['Claim Description']
- y=  df_train_rus['Coverage Code']
- X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.1,stratify=y,random_state=33)
- |
- |
- count_vector_step_1
- count_vectorizer_2
-  X_testtf = count_vector_step_1.transform(X_test)

NameError: name 'scikit' is not defined